<a href="https://colab.research.google.com/github/erfrits/girls_day_ds/blob/main/nlp_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
import numpy as np

# Nyelvi modellek

## Szózsákmodell (bag of words)

In [110]:
sentences = [
    "A kutya átfut a parkon.",
    "A macska átfut a parkon.",
    "A macska egeret fogott.",
    "A macska szereti a kutyát, nem az egeret.",
    "Az egeret szereti a macska, nem a kutyát."
]

In [118]:
clean_sentences = [s.replace(".", "").replace(",", "").lower() for s in sentences]
clean_sentences

['a kutya átfut a parkon',
 'a macska átfut a parkon',
 'a macska egeret fogott',
 'a macska szereti a kutyát nem az egeret',
 'az egeret szereti a macska nem a kutyát']

In [119]:
words = [w for s in clean_sentences for w in s.split(" ")]
words

['a',
 'kutya',
 'átfut',
 'a',
 'parkon',
 'a',
 'macska',
 'átfut',
 'a',
 'parkon',
 'a',
 'macska',
 'egeret',
 'fogott',
 'a',
 'macska',
 'szereti',
 'a',
 'kutyát',
 'nem',
 'az',
 'egeret',
 'az',
 'egeret',
 'szereti',
 'a',
 'macska',
 'nem',
 'a',
 'kutyát']

In [120]:
# szógyakoriság
from collections import Counter

c = Counter(words)
c.most_common(5)

[('a', 9), ('macska', 4), ('egeret', 3), ('átfut', 2), ('parkon', 2)]

In [121]:
# nem ismert szavakkal mi a helyzet?
c['zöld']

0

In [132]:
# reprezentáció
dictionary = list(set(words))  # duplikációk eltűntetése

def convert_senctence_to_bow(sentence):
  """
  Lefordítja a mondatot számvektorra szózsák módszer szerint
  """
  c_sentence = sentence.replace(".", "").replace(",", "").lower().split(" ")
  c = Counter(c_sentence)
  bow = {k: c[k] for k in dictionary}
  return bow


convert_senctence_to_bow("A kutya és a macska egeret fogott.")

{'az': 0,
 'parkon': 0,
 'kutyát': 0,
 'a': 2,
 'szereti': 0,
 'nem': 0,
 'egeret': 1,
 'fogott': 1,
 'kutya': 1,
 'macska': 1,
 'átfut': 0}

In [133]:
bow_of_sentences = [convert_senctence_to_bow(s) for s in sentences]

pd.DataFrame(bow_of_sentences, index=sentences)

,az,parkon,kutyát,a,szereti,nem,egeret,fogott,kutya,macska,átfut
A kutya átfut a parkon.,0,1,0,2,0,0,0,0,1,0,1
A macska átfut a parkon.,0,1,0,2,0,0,0,0,0,1,1
A macska egeret fogott.,0,0,0,1,0,0,1,1,0,1,0
"A macska szereti a kutyát, nem az egeret.",1,0,1,2,1,1,1,0,0,1,0
"Az egeret szereti a macska, nem a kutyát.",1,0,1,2,1,1,1,0,0,1,0


In [134]:
# hasonlóság = vektortávolság
def calculate_distances(text1, text2):

  bow1 = convert_senctence_to_bow(text1)
  bow2 = convert_senctence_to_bow(text2)
  vector1 = [bow1[k] for k in dictionary]
  vector2 = [bow2[k] for k in dictionary]

  first = np.array(vector1)
  second = np.array(vector2)

  # euklideszi távolság
  dist = np.linalg.norm(first - second)

  # cosine hasonlóság (két vektor által bezárt szög koszinusza)
  cos_sim = np.dot(first, second ) / (np.linalg.norm(first) * np.linalg.norm(second) + 1e-12)

  return (dist, cos_sim)

In [141]:
dist, cos_sim = calculate_distances(sentences[0], sentences[1])
print(sentences[0])
print(sentences[1])
print(f"Eukklédeszi távolság: {dist}")
print(f"Cosine hasonlóság: {cos_sim}")

A kutya átfut a parkon.
A macska átfut a parkon.
Eukklédeszi távolság: 1.4142135623730951
Cosine hasonlóság: 0.8571428571427345


In [142]:
dist, cos_sim = calculate_distances(sentences[0], sentences[2])
print(sentences[0])
print(sentences[1])
print(f"Eukklédeszi távolság: {dist}")
print(f"Cosine hasonlóság: {cos_sim}")

A kutya átfut a parkon.
A macska átfut a parkon.
Eukklédeszi távolság: 2.6457513110645907
Cosine hasonlóság: 0.37796447300915575


In [138]:
# probléma: nincs szórend vagy kontextus
dist, cos_sim = calculate_distances(
    "A macska szereti a kutyát, nem az egeret.",
    "A macska szereti az egeret, nem a kutyát."
  )
print(f"Eukklédeszi távolság: {dist}")
print(f"Cosine hasonlóság: {cos_sim}")

Eukklédeszi távolság: 0.0
Cosine hasonlóság: 0.9999999999998999


In [137]:
# probláma: csak azokat a szavakat ismeri, amikre megtanítottuk, minden más nullvektor => távolság = 1, cosine = 0
dist, cos_sim = calculate_distances("Tegnap havazott.", "A Himalája nagyon magas.")
print(f"Eukklédeszi távolság: {dist}")
print(f"Cosine hasonlóság: {cos_sim}")

Eukklédeszi távolság: 1.0
Cosine hasonlóság: 0.0


## Szóbeágyazások (word embeddings)

In [1]:
import gensim.downloader

In [3]:
%%time

wiki_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded
CPU times: user 38.6 s, sys: 1.61 s, total: 40.2 s
Wall time: 43.5 s


In [18]:
# Szóreprezentáció

wiki_vectors['apple']

array([ 0.52042 , -0.8314  ,  0.49961 ,  1.2893  ,  0.1151  ,  0.057521,
       -1.3753  , -0.97313 ,  0.18346 ,  0.47672 , -0.15112 ,  0.35532 ,
        0.25912 , -0.77857 ,  0.52181 ,  0.47695 , -1.4251  ,  0.858   ,
        0.59821 , -1.0903  ,  0.33574 , -0.60891 ,  0.41742 ,  0.21569 ,
       -0.07417 , -0.5822  , -0.4502  ,  0.17253 ,  0.16448 , -0.38413 ,
        2.3283  , -0.66682 , -0.58181 ,  0.74389 ,  0.095015, -0.47865 ,
       -0.84591 ,  0.38704 ,  0.23693 , -1.5523  ,  0.64802 , -0.16521 ,
       -1.4719  , -0.16224 ,  0.79857 ,  0.97391 ,  0.40027 , -0.21912 ,
       -0.30938 ,  0.26581 ], dtype=float32)

In [39]:
# Hány szó van a modellben?

f"{len(wiki_vectors):,}"

'400,000'

In [31]:
# Hol van az "apple"?

wiki_vectors.key_to_index['apple']

3292

In [44]:
np.random.seed = 2234

In [45]:
np.random.choice(wiki_vectors.index_to_key[3000:5000], 25, replace=False)

array(['hostage', 'accounting', 'controversy', 'wu', 'entirely',
       '500,000', 'rio', 'besides', 'complained', '53', 'roots',
       'chapter', '1950', 'sox', 'suburb', 'interesting', 'tibet',
       'touchdown', 'deeply', 'cambridge', 'feels', 'walking', 'pm',
       'medals', 'occasion'], dtype='<U18')

In [34]:
wiki_vectors['tiktok']

KeyError: "Key 'tiktok' not present"

### Szavak hasonlósága - szinonímák

In [19]:
similar_to_smile = wiki_vectors.most_similar(['smile'])
similar_to_smile

[('grin', 0.8619306683540344),
 ('smiles', 0.8397865295410156),
 ('eyes', 0.7954673767089844),
 ('smiling', 0.7861469388008118),
 ('laugh', 0.7707611322402954),
 ('touch', 0.7635201811790466),
 ('hug', 0.7596133351325989),
 ('bit', 0.7371944189071655),
 ('gentle', 0.7313948273658752),
 ('smirk', 0.7309232354164124)]

In [24]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


def plot_words(words):
  tsne = TSNE(n_components=2, random_state=0, perplexity=len(words)-1)
  vectors = [wiki_vectors[word] for word in words]
  Y = tsne.fit_transform(pd.DataFrame(vectors))
  plt.scatter(Y[:, 0], Y[:, 1])

  for label, x, y in zip(words, Y[:, 0], Y[:, 1]):
      plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")
  plt.show()

In [25]:
plot_words()

ValueError: perplexity must be less than n_samples

## Fontos, hogy milyen adatokon tanítunk

In [2]:
%%time

twitter_vectors = gensim.downloader.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded
CPU times: user 1min 53s, sys: 5.52 s, total: 1min 58s
Wall time: 2min 12s


In [20]:
similar_to_smile

[('grin', 0.8619306683540344),
 ('smiles', 0.8397865295410156),
 ('eyes', 0.7954673767089844),
 ('smiling', 0.7861469388008118),
 ('laugh', 0.7707611322402954),
 ('touch', 0.7635201811790466),
 ('hug', 0.7596133351325989),
 ('bit', 0.7371944189071655),
 ('gentle', 0.7313948273658752),
 ('smirk', 0.7309232354164124)]

In [6]:
twitter_vectors.most_similar(['smile'])

[('kiss', 0.8498159050941467),
 ('heart', 0.8433985710144043),
 ('eyes', 0.8414220809936523),
 ('hug', 0.8323259353637695),
 ('love', 0.827944278717041),
 ('smiling', 0.8259447813034058),
 ('you', 0.8204987645149231),
 ('always', 0.8151404857635498),
 ('hugs', 0.8127555847167969),
 ('your', 0.801873505115509)]

In [13]:
wiki_vectors.most_similar(['arms'])

[('weapons', 0.8018144369125366),
 ('hand', 0.7318101525306702),
 ('armed', 0.7299286127090454),
 ('carrying', 0.724751353263855),
 ('carry', 0.7231833338737488),
 ('cross', 0.7169591784477234),
 ('pressed', 0.7155767679214478),
 ('holding', 0.7145681977272034),
 ('military', 0.7131772041320801),
 ('crack', 0.7077354192733765)]

In [14]:
twitter_vectors.most_similar(['arms'])

[('hands', 0.886970043182373),
 ('neck', 0.8406134843826294),
 ('arm', 0.8387513160705566),
 ('chest', 0.8353148102760315),
 ('feet', 0.8316022753715515),
 ('legs', 0.8287435173988342),
 ('knees', 0.8170155882835388),
 ('hand', 0.8160897493362427),
 ('shoulders', 0.8130438327789307),
 ('fingers', 0.812550961971283)]